In [1]:
import os
import json
from pathlib import Path
import datetime
#from PIL import Image
#from PIL.ExifTags import TAGS
import numpy as np
from skimage import draw


In [2]:
def polygon_area_calculation(x_inputs, y_inputs):
    '''
    Function generate a list of all coordinates that are found within the polygon object.
    Generates a list of x and y coordinates of the same length (they are pairs). In order to
    access the coordinates, step through each list (x_coordiantes, y_coordinates) at the same rate
    ie. one for loop with len(x_coordinates)
    :param x_inputs: list of x coordinates from the annotated json file
    :param y_inputs: list of y coordinates from the annotated json file
    :return: x_coordinates, y_coordinates which hold the x and y, respectively, coordinates of the polygon.
    '''
    r = np.array(x_inputs)
    c = np.array(y_inputs)
    x_coordinates, y_coordinates = draw.polygon(r, c)
    return x_coordinates, y_coordinates


In [3]:
## Opening up meta.json file to figure out classes and class ids
with open("config.json") as json_file:
    content = json.load(json_file)
classes = [stuff for stuff in content['class_title_to_idx'] if stuff != 'bg']

## Class ids can be assigned in the order seen in classes. 'bg' will always be class id 0.
## Indexes should be starting at 1
mappedClasses = {c: i+1 for i, c in enumerate(classes)}

In [4]:
## JSON structure that will define the classes and ids to be used witin the file. This will be appended at the end
catg_repr = [{
            "id": v,
            "name": k,
            "supercategory": "type"
    } for k,v in mappedClasses.items()]

In [5]:
## Directories to pull annotations and images from 
ann_directory = Path('ann')
images_directory = Path('images')

In [7]:
## Loading up all the JSON filenames and their content into lists, this will get all File I/O initially
filenames = []
json_content = []
for annotation_files in os.listdir(ann_directory):
    image_filename = annotation_files[:-5]
    with open(ann_directory / annotation_files) as fp:
        content = json.load(fp)
    filenames.append(image_filename)
    json_content.append(content)

print("Lenght filenames: {} Length json_content: {}".format(len(filenames), len(json_content)))
    

Lenght filenames: 10 Length json_content: 10


In [8]:
## Dictionary holding general metadata about the project 
info = [{
    "description": "Converting Supervisely JSON format polygons into COCO. (Work in progress)",
    "url": "null",
    "version": "1.0",
    "year": datetime.datetime.now().year,
    "contributor": "Original Script by Caio Marcellos, Modified by Sai Peri to work properly.",
    "date_created": datetime.datetime.now().isoformat()
}]

In [9]:
## Dictionary holding general metadata about the project 
licenses = [{
    "url": "null",
    "id": 1,
    "name": "Name of License"
}]

In [10]:
images = []
for i, files in enumerate(filenames):
    ## Generate absolute filepath for the image corresponding to the JSON file
    image_filepath = images_directory / files
    image_filepath = image_filepath.absolute()
    
    ## Open up image and get metadata about height, width, date_captured
    ## Unfortunately our dataset does not have proper metadata to extract this type of information
    #image = Image.open(image_filepath)
    #exif_data = image.getexif()

    images.append({"license": 1,
                    "file_name": str(image_filepath),
                    "height": 512,
                    "width": 640,
                    "date_captured": datetime.datetime.now().isoformat(),
                    "id": i})

In [11]:
## Generate annotations for each of the objects detected within supervisely JSON file
## Currently only dumps coordinates from exterior into segmentation. 
## TODO: Incorperate holes, interior, and rebuild contour based on new coordiantes
## TODO: Train/val tagging
annotations = []
annotation_id = 0
for i, json_files in enumerate(json_content):
    objects = json_files['objects']
    for data in objects:
        classTitle = data['classTitle']
        class_id = mappedClasses.get(classTitle)
        #print("classTitle: {} | class_id: {}".format(classTitle, class_id))
        exterior = data['points']['exterior']
        segmentation = []
        x = []
        y = []
        for poly_points in exterior:
            x.append(poly_points[1])
            y.append(poly_points[0])
            segmentation.append(poly_points[1])
            segmentation.append(poly_points[0])
        
        bbox_coordinates = [min(x), min(y), max(x), max(y)]

        ## Generating all coordinates within polygon shape to count area. Caio's implementation was not doing calculation
        ## based on each class but rather just kept appending
        x_coord, y_coord = polygon_area_calculation(x, y)
        #print("Filename: {} | Classname: {} | Area: {}".format(filenames[i], classTitle, len(x_coord)))
        
        ## Appending annotation
        annotations.append({
            "segmentation": [segmentation],
            "area": len(x_coord),
            "iscrowd": 0,
            "image_id": i,
            "bbox": bbox_coordinates,
            "category_id": class_id,
            "id": annotation_id
        })
        annotation_id += 1
        #print(annotations)

        


In [12]:
coco_fmt = {
    "info": info,
    "images": images,
    "annotations": annotations,
    "licenses": licenses,
    "categories": catg_repr
}

In [13]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj): #pylint: disable=method-hidden
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

In [16]:
with open(os.curdir + "\\instances_test2014.json", "w") as fp:
    json.dump(coco_fmt, fp, cls=NpEncoder)